*langchain*

In [1]:
!pip install requests-cache


In [2]:
import requests
import requests_cache
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Cargar las variables de entorno
load_dotenv()
api_key = os.getenv('NEWS_API_KEY')

# Configurar el caché con una expiración de 1 día
requests_cache.install_cache('news_cache', expire_after=timedelta(days=1))

# Lista de empresas
empresas = [
    "Altitude Investment Management",
    "Alinga Capital",
    "Aurora Cannabis",
    "acreageholdings",
    "4 Front Ventures",
    "Barenberg Capital Partners",
    "Bayline Capital Partners",
    "Big Rock",
    "Bullpen Capital",
    "cannaangelsllc.com",
    "cannahedge.com",
    "Canalis Capital",
    "Cannabis Capital",
    "Canopy Rivers",
    "canopyboulder",
    "Charlottes Web Holdings",
    "casaverdecapital",
    "canopygrowth",
    "Compass Cannabis Clinic",
    "Constellation Brands",
    "Craft Ventures",
    "Cusp Capital",
    "Capital 8 Group",
    "First Trust Portafolio",
    "FundRx",
    "KES7 Capital",
    "Krauter Group",
    "KushCo Holdings",
    "LaMarch Capital",
    "Lerer Hippeau Ventures",
    "Mainline Investment Partners",
    "Mazakali",
    "Medmen",
    "Mentor Capital",
    "Merida Capital Partners",
    "Miracle Mile Advisors",
    "Montegra Capital",
    "muse capital",
    "Navy Capital",
    "Nosara Capital Growthinc",
    "Phyto Partners",
    "Privateer Holdings",
    "Rose Capital",
    "Salveo Capital",
    "Green Acre Capital",
    "Green Growth Investments",
    "Green Leaf Investments",
    "HIGH TIMES",
    "Initiative Capital",
    "Supreme Cannabis",
    "The Arcview Group",
    "Tress Capital",
    "High Tide",
    "Crowe",
    "Flower Hire",
    "Primary Growth Partners",
    "CHICAGO ATLANTIC GROUP",
    "GREEN SPACE ACCOUNTING",
    "Merida Capital Holdings",
    "Fluence",
    "Woodcrick Ventures"
]

# Endpoint de la API
url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 7 días)
from_date = (datetime.now() - timedelta(days=10)).strftime('%Y-%m-%d')

news_data = {}

for empresa in empresas:
    print(f"Buscando noticias para: {empresa}")
    params = {
        'q': empresa,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'language': 'en',  # Idioma de las noticias
        'from': from_date  # Fecha desde la cual buscar noticias
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        news_data[empresa] = []
        for article in articles:
            title = article['title']
            link = article['url']
            snippet = article['description']
            published_at = article['publishedAt']
            news_data[empresa].append(
                {'title': title, 'link': link, 'snippet': snippet, 'published_at': published_at})
        print(f"Encontradas {len(articles)} noticias para {empresa}")
    except requests.exceptions.RequestException as e:
        news_data[empresa] = f"Error fetching data: {str(e)}"
        print(f"Error al buscar noticias para {empresa}: {str(e)}")

# No guardar en archivo JSON, continuar con el procesamiento en memoria


Buscando noticias para: Altitude Investment Management
Encontradas 15 noticias para Altitude Investment Management
Buscando noticias para: Alinga Capital
Encontradas 0 noticias para Alinga Capital
Buscando noticias para: Aurora Cannabis
Encontradas 7 noticias para Aurora Cannabis
Buscando noticias para: acreageholdings
Encontradas 0 noticias para acreageholdings
Buscando noticias para: 4 Front Ventures
Encontradas 23 noticias para 4 Front Ventures
Buscando noticias para: Barenberg Capital Partners
Encontradas 0 noticias para Barenberg Capital Partners
Buscando noticias para: Bayline Capital Partners
Encontradas 0 noticias para Bayline Capital Partners
Buscando noticias para: Big Rock
Encontradas 100 noticias para Big Rock
Buscando noticias para: Bullpen Capital
Encontradas 2 noticias para Bullpen Capital
Buscando noticias para: cannaangelsllc.com
Encontradas 0 noticias para cannaangelsllc.com
Buscando noticias para: cannahedge.com
Encontradas 0 noticias para cannahedge.com
Buscando not

In [3]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pprint

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Procesar los datos en memoria
extracted_elements = []

for company, articles in news_data.items():
    if isinstance(articles, list):
        for article in articles:
            title = article.get('title', 'No title')
            snippet = article.get('snippet', 'No snippet')
            link = article.get('link', 'No link')
            extracted_elements.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")

# Filtrar empresas sin noticias
filtered_elements = [element for element in extracted_elements if 'Title: No title'or'snippet:Nosi' not in element]

# Crear documentos para Langchain
documents = [Document(page_content=element) for element in filtered_elements]


# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.4, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

/home/ozz/anaconda3/envs/envopenai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/home/ozz/anaconda3/envs/envopenai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:

# Lista de preguntas para identificar similitudes y repetición de temas entre noticias
questions = [
    "¿Cuáles son los temas más mencionados en las noticias recientes sobre la empresa?",
    "¿Qué titulares de noticias se repiten entre los diferentes artículos sobre la empresa",
    "¿Qué términos o frases aparecen con mayor frecuencia en los titulares de las noticias sobre la empresa?",
    "¿Cuáles son los impactos y consecuencias más destacados mencionados en las noticias sobre la empresa?",
    "¿Qué personas o entidades son mencionadas con mayor frecuencia en las noticias sobre la empresa?",
]


# Almacenar las respuestas en una lista
responses = []

for empresa in news_data:
    print(f"Analizando preguntas para: {empresa}")
    for j, query in enumerate(questions):
        full_query = f"{query} {empresa}"
        response = qa_chain({"query": full_query})
        responses.append({
            "empresa": empresa,
            "query": query,
            "response": response["result"]
        })
        print(f"Pregunta {j+1}/{len(questions)} analizada para {empresa}")



/home/ozz/anaconda3/envs/envopenai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Analizando preguntas para: Altitude Investment Management
Pregunta 1/5 analizada para Altitude Investment Management
Pregunta 2/5 analizada para Altitude Investment Management
Pregunta 3/5 analizada para Altitude Investment Management
Pregunta 4/5 analizada para Altitude Investment Management
Pregunta 5/5 analizada para Altitude Investment Management
Analizando preguntas para: Alinga Capital
Pregunta 1/5 analizada para Alinga Capital
Pregunta 2/5 analizada para Alinga Capital
Pregunta 3/5 analizada para Alinga Capital
Pregunta 4/5 analizada para Alinga Capital
Pregunta 5/5 analizada para Alinga Capital
Analizando preguntas para: Aurora Cannabis
Pregunta 1/5 analizada para Aurora Cannabis
Pregunta 2/5 analizada para Aurora Cannabis
Pregunta 3/5 analizada para Aurora Cannabis
Pregunta 4/5 analizada para Aurora Cannabis
Pregunta 5/5 analizada para Aurora Cannabis
Analizando preguntas para: acreageholdings
Pregunta 1/5 analizada para acreageholdings
Pregunta 2/5 analizada para acreageholdi

In [5]:
# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=10)
pp.pprint(responses)

[         {         'empresa': 'Altitude Investment Management',
                    'query': '¿Cuáles son los temas más mencionados en las '
                             'noticias recientes sobre la empresa?',
                    'response': 'Lo siento, no tengo información específica '
                                'sobre los temas más mencionados en las '
                                'noticias recientes sobre Altitude Investment '
                                'Management. Te recomendaría buscar en fuentes '
                                'de noticias financieras o en el sitio web de '
                                'la empresa para obtener información '
                                'actualizada sobre sus actividades y temas '
                                'relevantes.'},
          {         'empresa': 'Altitude Investment Management',
                    'query': '¿Qué titulares de noticias se repiten entre los '
                             'diferentes artículos sobr

In [6]:
texto = ''' Crowe mencionan '
                                'a Vaughn Crowe, un nativo de Newark, NJ,'''

In [7]:


texto_limpio = ' '.join(texto.split())

print(texto_limpio)

Crowe mencionan ' 'a Vaughn Crowe, un nativo de Newark, NJ,


Cannabis en general

In [8]:
import requests
import requests_cache
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Cargar las variables de entorno

api_key = os.getenv('NEWS_API_KEY')

# Configurar el caché con una expiración de 1 día
requests_cache.install_cache('news_cache', expire_after=timedelta(days=1))

# Lista de temas relacionados con cannabis
temas = [
    "Cannabis legalization",
    "Medical marijuana",
    "Cannabis industry",
    "Cannabis investment",
    "Cannabis stocks",
    "Cannabis research",
    "Cannabis news",
    "Cannabis policy",
    "Cannabis business",
    "Cannabis culture",
    "Psychedelics research",
    "Psychedelics industry",
    "Psychedelics investment",
    "Psychedelics news",
    "Psychedelics policy",
    "Psychedelics business",
    "Psychedelics culture"
]

# Endpoint de la API
url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 7 días)
from_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')

news_data = {}

for tema in temas:
    print(f"Buscando noticias para: {tema}")
    params = {
        'q': tema,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'language': 'en',  # Idioma de las noticias
        'from': from_date  # Fecha desde la cual buscar noticias
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        if articles:
            news_data[tema] = []
            for article in articles:
                title = article['title']
                link = article['url']
                snippet = article['description']
                published_at = article['publishedAt']
                news_data[tema].append(
                    {'title': title, 'link': link, 'snippet': snippet, 'published_at': published_at})
            print(f"Encontradas {len(articles)} noticias para {tema}")
        else:
            print(f"No se encontraron noticias para {tema}")
    except requests.exceptions.RequestException as e:
        print(f"Error al buscar noticias para {tema}: {str(e)}")

# Filtrar temas sin noticias
news_data = {tema: articles for tema, articles in news_data.items() if articles}

# No guardar en archivo JSON, continuar con el procesamiento en memoria


Buscando noticias para: Cannabis legalization
Encontradas 19 noticias para Cannabis legalization
Buscando noticias para: Medical marijuana
Encontradas 49 noticias para Medical marijuana
Buscando noticias para: Cannabis industry
Encontradas 81 noticias para Cannabis industry
Buscando noticias para: Cannabis investment
Encontradas 30 noticias para Cannabis investment
Buscando noticias para: Cannabis stocks
Encontradas 26 noticias para Cannabis stocks
Buscando noticias para: Cannabis research
Encontradas 59 noticias para Cannabis research
Buscando noticias para: Cannabis news
Encontradas 100 noticias para Cannabis news
Buscando noticias para: Cannabis policy
Encontradas 36 noticias para Cannabis policy
Buscando noticias para: Cannabis business
Encontradas 86 noticias para Cannabis business
Buscando noticias para: Cannabis culture
Encontradas 17 noticias para Cannabis culture
Buscando noticias para: Psychedelics research
Encontradas 12 noticias para Psychedelics research
Buscando noticias 

In [9]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pprint

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Procesar los datos en memoria
extracted_elements = []

for tema, articles in news_data.items():
    for article in articles:
        title = article.get('title', 'No title')
        snippet = article.get('snippet', 'No snippet')
        link = article.get('link', 'No link')
        extracted_elements.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")

# Crear documentos para Langchain
documents = [Document(page_content=element) for element in extracted_elements]

# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [14]:
questions = [
    "Cuales son los temas mas populares y mencionados en las noticias recientes sobre el tema, menciona los titulares",
    "¿Cuáles son los temas más mencionados en las noticias recientes sobre el tema?",
    "¿Qué titulares de noticias se repiten entre los diferentes artículos sobre el tema?",
    "¿Qué términos o frases aparecen con mayor frecuencia en los titulares de las noticias sobre el tema?",
    "¿Cuáles son los impactos y consecuencias más destacados mencionados en las noticias sobre el tema?",
    "¿Qué personas o entidades son mencionadas con mayor frecuencia en las noticias sobre el tema?"
]

# Almacenar las respuestas en una lista
responses = []

for tema in news_data:
    print(f"Analizando preguntas para: {tema}")
    for j, query in enumerate(questions):
        full_query = f"{query} {tema}"
        response = qa_chain({"query": full_query})
        responses.append({
            "tema": tema,
            "query": query,
            "response": response["result"]
        })
        print(f"Pregunta {j+1}/{len(questions)} analizada para {tema}")



Analizando preguntas para: Cannabis legalization
Pregunta 1/6 analizada para Cannabis legalization
Pregunta 2/6 analizada para Cannabis legalization
Pregunta 3/6 analizada para Cannabis legalization
Pregunta 4/6 analizada para Cannabis legalization
Pregunta 5/6 analizada para Cannabis legalization
Pregunta 6/6 analizada para Cannabis legalization
Analizando preguntas para: Medical marijuana
Pregunta 1/6 analizada para Medical marijuana
Pregunta 2/6 analizada para Medical marijuana
Pregunta 3/6 analizada para Medical marijuana
Pregunta 4/6 analizada para Medical marijuana
Pregunta 5/6 analizada para Medical marijuana
Pregunta 6/6 analizada para Medical marijuana
Analizando preguntas para: Cannabis industry
Pregunta 1/6 analizada para Cannabis industry
Pregunta 2/6 analizada para Cannabis industry
Pregunta 3/6 analizada para Cannabis industry
Pregunta 4/6 analizada para Cannabis industry
Pregunta 5/6 analizada para Cannabis industry
Pregunta 6/6 analizada para Cannabis industry
Analizand

In [15]:
# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=8)
pp.pprint(responses)

[       {       'query': 'Cuales son los temas mas populares y mencionados en '
                         'las noticias recientes sobre el tema, menciona los '
                         'titulares',
                'response': 'No tengo información específica sobre los temas '
                            'más populares y mencionados en las noticias '
                            'recientes sobre la legalización del cannabis.',
                'tema': 'Cannabis legalization'},
        {       'query': '¿Cuáles son los temas más mencionados en las '
                         'noticias recientes sobre el tema?',
                'response': 'Los temas más mencionados en las noticias '
                            'recientes sobre la legalización del cannabis '
                            'incluyen la posibilidad de rescheduling del '
                            'cannabis como una droga de Clase 3 en los Estados '
                            'Unidos, críticas al enfoque del gobierno '
          

In [12]:
texto = ''' Some moms are microdosing on mushrooms, '
                            'touting the benefits – but risks exist, say '
                            'doctors. '''

texto_limpio = ' '.join(texto.split())

print(texto_limpio)


Some moms are microdosing on mushrooms, ' 'touting the benefits – but risks exist, say ' 'doctors.


In [13]:
# Cálculo del costo total basado en las nuevas áreas y el precio por metro cuadrado

# Áreas en metros cuadrados
centro_convenciones = 100
restaurante = 50
zonas_descarga = 50
oficinas = 50

# Precios por metro cuadrado en COP
precio_minimo_m2 = 50000
precio_maximo_m2 = 60000

# Cálculo del área total
area_total = centro_convenciones + restaurante + zonas_descarga + oficinas

# Cálculo del costo total basado en el precio por metro cuadrado
costo_total_minimo = area_total * precio_minimo_m2
costo_total_maximo = area_total * precio_maximo_m2

costo_total_minimo, costo_total_maximo, area_total

(12500000, 15000000, 250)